# <u>Sujet 1 : Evaluation de deux plateformes open source d'analyse linguistique</u>

## <u> I - Evaluation de l'analyse morpho-syntaxique </u>

##### 1) Utiliser le corpus annoté « pos_reference.txt.lima » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « pos_test.txt  ».

In [3]:
def getText(fileIn, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    for line in fileI:

        tuples = line.split()
        if tuples:
            # On souhaite récupérer l'entièreté de la ligne à l'exception du dernier mot
            # qui est l'annotation
            for i in range(len(tuples)-1):
                fileO.write(tuples[i]+" ")
        else:
            # Si on arrive à une ligne vide on saute une ligne c'est un changement de phrase 
            fileO.write("\n")

    fileI.close()
    fileO.close()


In [4]:
fileIn = "pos_reference.txt.lima"
fileOut = "pos_test.txt"

getText(fileIn,fileOut)


##### 2) Convertir  les  tags  du  corpus  annoté  « pos_reference.txt.lima »  en  tags  universels  et sauvegarder le résultat dans le fichier « pos_reference.txt.univ ». Il faut utiliser les deux tables de correspondance « POSTags_LIMA_PTB_Linux.txt » et « POSTags_PTB_Universal_Linux.txt » pour réaliser cette transformation (LIMA => PTB => Universal). 
 

In [5]:
def setTagUniv(fileIn, fileRef1,fileRef2, fileOut):
    fileI = open(fileIn,'r')
    fileR1 = open(fileRef1,'r')
    fileR2 = open(fileRef2,'r')
    fileO = open(fileOut,'w')

    my_dico1 = {}
    my_dico2 = {}

    #on crée deux dictionnaires à partir des tags de reference
    for line in fileR1:
        tuples = line.split()
        my_dico1[tuples[0]] = tuples[1]
    fileR1.close()

    for line in fileR2:
        tuples = line.split()
        my_dico2[tuples[0]] = tuples[1]
    fileR2.close()

    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            for i in range(len(tup)-1):
                    fileO.write(tup[i]+" ")
            temp = my_dico1[tup[len(tup)-1]]

            fileO.write('\t'+ my_dico2[temp])
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [6]:
fileIn = "pos_reference.txt.lima"
fileRef1 = "POSTags_LIMA_PTB_Linux.txt"
fileRef2 = "POSTags_PTB_Universal_Linux.txt"
fileOut = "pos_reference.txt.univ"


setTagUniv(fileIn,fileRef1,fileRef2,fileOut)


##### 3) Lancer les deux POS taggers sur le fichier « pos_test.txt ». Les résultats doivent avoir le format du  corpus  annoté  « pos_reference.txt.lima »  (2  colonnes  séparées  par  une  tabulation)  et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford 
-  pos_test.txt.pos.nltk 

In [7]:
import nltk
from nltk.tokenize import word_tokenize

def nltkTokenize(fileIn,fileOut):
    
    # Ouvrir le fichier en lecture seule
    file = open(fileIn, "r")
    fileO = open(fileOut, "w")
    for line in file:
        
        text = word_tokenize(line)
    
        tokens_tag = nltk.pos_tag(text)
        
        for pair in tokens_tag :
            fileO.write('\t'.join(pair))  
            fileO.write('\n')
    fileO.close()
    file.close()   

Ligne de commande à taper dans le terminal linux depuis le dossier stanford-postagger

./stanford-postagger.sh models/english-left3words-distsim.tagger ../pos_test.txt > ../pos_test.txt.pos.stanford

In [8]:
fileIn = "pos_test.txt"
fileOutNltk = "pos_test.txt.pos.nltk"
fileOutStan = "pos_test.txt.pos.stanford"

nltkTokenize(fileIn,fileOutNltk)


##### 4) Convertir les résultats des deux POS taggers en utilisant les étiquettes universelles (Annexe 1). Il faut utiliser la table de correspondance « POSTags_PTB_Universal_Linux.txt » pour réaliser cette  transformation  (PTB => Universal).  Les  résultats  de  cette  conversion  doivent  être sauvegardés respectivement dans les fichiers suivants : 
-  pos_test.txt.pos.stanford.univ 
-  pos_test.txt.pos.nltk.univ

In [9]:
# Création du dictionnaire PTB vers Universal
my_dico = {}

#on crée un dictionnaire à partir des tags de reference
fileR = open(fileRef2,'r')
for line in fileR:
    tuples = line.split()
    my_dico[tuples[0]] = tuples[1]
fileR.close()




In [10]:
def setTagPTBUnivNltk(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')

    
    # modification des tags vers les tags univ
    for l in fileI:
        tup = l.split()
        if tup:
            # on écrit le groupe de mots initial
            fileO.write(tup[0]+'\t'+ dico[tup[1]])
           
            fileO.write('\n')
        else:
            fileO.write("\n")

    fileI.close()
    fileO.close()

In [11]:
setTagPTBUnivNltk(fileOutNltk, my_dico, "pos_test.txt.pos.nltk.univ")

##### Ajout d'étiquettes dans le fichier PTB vers Univ car nous avons constater un manque pour les étiquettes suivantes :
-  $ => CUR
-  `` => .
-  '' => .

In [17]:
def setTagPTBUnivStan(fileIn, dico, fileOut):
    fileI = open(fileIn,'r')
    fileO = open(fileOut,'w')
    checkTag = 0
    w = ""
    for l in fileI:
        tup = l.split()
        for word in tup:
            for letter in word:
                if(letter == "_"):
                    fileO.write("\t")
                    checkTag = 1
                if(checkTag == 1 and letter != "_"):
                    w = w+letter
                if(letter != "_" and checkTag == 0):
                    fileO.write(letter)
            fileO.write(" "+dico[w]+" ")
            print(dico[w])
            fileO.write('\n')
            checkTag = 0
            w = ""
    fileI.close()
    fileO.close()   
    

In [18]:
setTagPTBUnivStan(fileOutStan,my_dico,"pos_test.txt.pos.stanford.univ")

NOUN
NOUN
.
NUM
NOUN
ADJ
.
VERB
VERB
DET
NOUN
ADP
DET
ADJ
NOUN
NOUN
NUM
.
NOUN
NOUN
VERB
NOUN
ADP
NOUN
NOUN
.
DET
ADJ
NOUN
NOUN
.
NOUN
NOUN
.
NUM
NOUN
ADJ
CONJ
ADJ
NOUN
ADP
NOUN
NOUN
NOUN
NOUN
.
VERB
VERB
DET
ADJ
NOUN
ADP
DET
ADJ
ADJ
NOUN
.
DET
NOUN
ADP
NOUN
ADV
VERB
PRT
VERB
NOUN
NOUN
NOUN
VERB
VERB
DET
ADJ
NOUN
ADP
NOUN
NOUN
ADP
DET
NOUN
ADP
NOUN
VERB
PRT
PRON
ADJ
ADP
NUM
NOUN
ADV
.
NOUN
VERB
.
DET
NOUN
NOUN
.
NOUN
.
VERB
ADV
ADJ
ADP
PRON
VERB
DET
NOUN
.
ADP
ADV
ADJ
NOUN
PRT
PRON
VERB
NOUN
DET
VERB
PRT
NOUN
ADV
.
NOUN
VERB
.
NOUN
NOUN
.
DET
NOUN
ADP
NOUN
NOUN
.
VERB
NOUN
NOUN
DET
VERB
NOUN
NOUN
.
VERB
VERB
NOUN
ADP
PRON
NOUN
NOUN
NOUN
ADP
NUM
.
ADP
ADJ
NOUN
VERB
VERB
ADV
ADP
DET
NOUN
ADV
.
DET
ADJ
NOUN
VERB
ADP
NOUN
PRT
NOUN
NOUN
NOUN
ADP
NOUN
.
DET
NOUN
ADJ
PRT
VERB
ADJ
NOUN
PRT
DET
NOUN
.
DET
NOUN
NOUN
VERB
.
.
DET
VERB
DET
ADJ
NOUN
.
PRON
VERB
VERB
ADP
NOUN
ADP
ADP
NOUN
VERB
ADP
NOUN
VERB
DET
ADJ
NOUN
.
DT
VERB
DET
NOUN
ADP
PRON
NOUN
ADV
.
.
DET
NOUN
CONJ
DET
NOUN
PRON
VERB
DET
NO

KeyError: ''

##### 5) Lancer l’évaluation des deux POS taggers. Pour réaliser cette évaluation, il faut supprimer la ligne vide (séparant les phrases) dans le de fichier de référence « pos_reference.txt.univ » : 
-  python evaluate.py pos_test.txt.pos.stanford.univ pos_reference.txt.univ 
-  python evaluate.py pos_test.txt.pos.nltk.univ pos_reference.txt.univ 

##### 6) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux POS taggers. 

## <u> II. Evaluation de la reconnaissance d’entités nommées </u>

##### 1) Utiliser le corpus annoté « ne_reference.txt.conll » pour extraire les phrases ayant servi pour produire ce corpus annoté et sauvegarder le résultat dans le fichier « ne_test.txt ». Dans ce corpus, une ligne vide indique la fin de la phrase courante. 
 

##### 2) Lancer les deux NE recognizers sur le fichier « ne_test.txt ». Les résultats doivent avoir le format  du  corpus  annoté  « ne_reference.txt.conll »  (2  colonnes  séparées  par  une tabulation) et doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford 
-  ne_test.txt.ne.nltk 

##### 3) Convertir  les  résultats  des  deux  NE  recognizers  en  utilisant  les  étiquettes  CoNLL-2003 (https://www.clips.uantwerpen.be/conll2003/ner/  -Annexe  2-).  Les  résultats  de  cette conversion doivent être sauvegardés respectivement dans les fichiers suivants : 
-  ne_test.txt.ne.stanford.conll 
-  ne_test.txt.ne.nltk.conll 

##### 4) Lancer  l’évaluation  des  deux  NE  recognizers.  Pour  réaliser  cette  évaluation,  il  faut supprimer  la  ligne  vide  (séparant  les  phrases)  dans  le  de  fichier  de  référence « ne_reference.txt.conll »: 
-  python evaluate.py ne_test.txt.ne.stanford.conll ne_reference.txt.conll 
-  python evaluate.py ne_test.txt.ne.nltk.conll ne_reference.txt.conll 

##### 5) Quelles conclusions vous pouvez avoir à partir des résultats d’évaluation des deux NE recognizers.